In [1]:
import pandas as pd
import numpy as np
import io
import tensorflow as tf

In [2]:
test=pd.read_csv('test_emoji.csv')

In [3]:
test

,sentences,emogi
0,I want to eat,4
1,he did not answer,3
2,he got a raise,2
3,she got me a present,0
4,ha ha ha it was so funny,2
5,he is a good friend,0
6,I am upset,0
7,We had such a lovely dinner tonight,0
8,where is the food,4
9,Stop making this joke ha ha ha,2


In [4]:
test_X=test['sentences'].to_numpy()
test_Y=test['emogi'].to_numpy()

In [5]:
value=[]
embeddings_index = {}
with io.open('glove.6B.50d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        value.append(values)
        word = values[0]
        coefs = np.asarray(values[1:],dtype='float32')
        embeddings_index[word] = coefs

In [6]:
word_to_index={}
index_to_word={}
for i,v in enumerate(value):
    word_to_index[v[0]]=i
    index_to_word[i]=v[0]

In [7]:
def sentences_to_indices(X, word_to_index, max_len):
    m = X.shape[0] 
    X_indices = np.zeros([m,max_len])
    for i in range(m): 
        sentence_words=X[i].lower().split()
        j=0
        for w in sentence_words:
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
                j =  j+1
    return X_indices

In [8]:
X_test=sentences_to_indices(test_X, word_to_index, 10)

In [9]:
X_test[0]

array([  41.,  303.,    4., 3623.,    0.,    0.,    0.,    0.,    0.,
          0.])

In [10]:
test_X[0]

'I want to eat'

In [11]:
model2 = tf.keras.models.load_model("./emogi_suggestor_model")#, options=localhost_save_option)

In [12]:
model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding (Embedding)       (None, 10, 50)            20000050  
                                                                 
 lstm (LSTM)                 (None, 10, 128)           91648     
                                                                 
 dropout (Dropout)           (None, 10, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 25)                3225  

In [13]:
y=model2.predict(X_test)

2/2 [==============================] - 3s 8ms/step


In [15]:
y=np.argmax(y,axis=1)

In [16]:
y

array([4, 3, 2, 0, 2, 2, 3, 2, 4, 2, 1, 2, 0, 2, 1, 3, 2, 2, 1, 2, 0, 0,
       4, 0, 3, 3, 2, 0, 3, 2, 0, 1, 0, 2, 0, 1, 2, 3, 4, 2, 1, 0, 0, 1,
       2, 0, 2, 2, 2, 1, 3, 0, 3, 2, 2, 4], dtype=int64)

In [18]:
test_Y

array([4, 3, 2, 0, 2, 0, 0, 0, 4, 2, 1, 3, 3, 2, 1, 3, 3, 2, 3, 4, 0, 2,
       4, 3, 3, 3, 3, 0, 2, 0, 0, 1, 3, 2, 0, 1, 2, 4, 4, 2, 2, 2, 0, 1,
       2, 0, 2, 2, 2, 3, 3, 0, 3, 2, 2, 3], dtype=int64)

In [23]:
z=(((test_Y-y)==0)+0)
print(z)

[1 1 1 1 1 0 0 0 1 1 1 0 0 1 1 1 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 1 0 1 1 1 1
 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0]


In [22]:
z1=np.sum(z)
print(z1/z.shape[0])

0.6607142857142857
